# Recommender

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

# Recommender

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()
With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />
Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)
data.head()
data.describe().show()
We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)
Now let's see hwo the model performed!
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.show()
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))
The RMSE described our error in terms of the stars rating column.
So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()
reccomendations = model.transform(single_user)
reccomendations.orderBy('prediction',ascending=False).show()
# Great Job!

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [7]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [8]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see how the model performed!

In [9]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    26|  3.9472167|
|     31|   1.0|    13|  0.5823611|
|     31|   1.0|     5|  2.4656975|
|     31|   3.0|     8|  5.0220237|
|     85|   1.0|    23| -1.9633777|
|     85|   3.0|    21|-0.30704117|
|     65|   1.0|    16|  1.2394556|
|     65|   2.0|     3|  0.9690968|
|     65|   2.0|    15| -0.2741957|
|     65|   5.0|    23|-0.44085988|
|     65|   1.0|     2|  0.7381696|
|     53|   1.0|     6|  0.9201523|
|     78|   1.0|    17| 0.98552036|
|     78|   1.0|     8|  1.5784727|
|     34|   1.0|    28|  2.4415681|
|     34|   1.0|    15|  2.5946007|
|     34|   1.0|    17|  1.9468824|
|     34|   4.0|     2| 0.09802568|
|     81|   1.0|     6|  0.9919184|
|     81|   2.0|     5|  3.5530796|
+-------+------+------+-----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 2.1161863484094554


The RMSE described our error in terms of the stars rating column.

So now that I have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [13]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [14]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     16|    11|
|     18|    11|
|     19|    11|
|     21|    11|
|     23|    11|
|     39|    11|
|     45|    11|
|     50|    11|
|     62|    11|
|     67|    11|
|     77|    11|
+-------+------+



In [15]:
reccomendations = model.transform(single_user)

In [16]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     23|    11|   4.054081|
|     50|    11|  3.2830575|
|     16|    11|  2.4960592|
|     67|    11|   2.432409|
|     62|    11|  2.3805099|
|     39|    11|  1.9306948|
|     45|    11|  1.7706994|
|     77|    11|  1.6215423|
|     18|    11|  1.4315107|
|     19|    11|  1.2213502|
|      6|    11|  1.0572225|
|     21|    11|-0.06694201|
+-------+------+-----------+

